# Runs Reddit API to collect posts containing a search term, and returns post with comments on it

### Import the Reddit credentials and modules required

In [2]:
from redditcredentials import CLIENT_ID, CLIENT_SECRET, USERNAME_REDDIT, PASSWORD_REDDIT
import requests

### Setup the request for Reddit API, OAuth authentication

In [3]:
# Authorisation code from https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c, as tried what API github suggested and did not work
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET)

data = {'grant_type': 'password',
        'username': USERNAME_REDDIT,
        'password': PASSWORD_REDDIT}

headers = {'User-Agent': 'TopicSentimentAcrossSN/0.0.1'}

res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

TOKEN = res.json()['access_token']

headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

### Set search term and create the dictionary to hold posts

In [21]:
search_term = 'Nicola Sturgeon'
reddit_posts = [] #{}

### Function to get the comments from the posts

In [22]:
def getCommentsFromPosts(post_id, post_title):
        comment_results = requests.get("https://oauth.reddit.com/r/all/comments/" + str(post_id),
                                headers=headers, params={'limit': '25'})

        print("PRINTING COMMENT RESULTS FOR THE TOPIC: \n")

        comments = comment_results.json()[1]['data']['children'][:-1]   #slices last value as it is just list of id's
        all_comments = []

        reddit_posts.append(post_title) #added

        for comment in comments:
                comment_text = comment['data']['body']
                reddit_posts.append(comment_text)
                #all_comments += [comment_text]
        
        #reddit_posts.append(all_comments)#reddit_posts[post_title] = all_comments

### Find results on search term from reddit posts, and collect comments containing that topic

In [23]:
results = requests.get("https://oauth.reddit.com/r/all/search", 
                headers=headers, params={'limit': '25', 'q':{search_term}, 'sort':'new'})

print("PRINTING SEARCH RESULTS FOR THE TOPIC: \n")

posts = results.json()['data']['children']

for post in posts:
        post_title = post['data']['title']
        print(post_title)
        post_id = post['data']['id']
        getCommentsFromPosts(post_id, post_title)

PRINTING SEARCH RESULTS FOR THE TOPIC: 

COP26: Greta Thunberg tells protest that COP26 has been a failure
PRINTING COMMENT RESULTS FOR THE TOPIC: 

First Minister Nicola sturgeon is delighted to have received, on behalf of the people of Scotland, a gift from President Biden. She was so pleased to welcome President Biden to Glasgow, and looks forward to him returning to Scotland in the future. COP 26
PRINTING COMMENT RESULTS FOR THE TOPIC: 

Scottish First Minister Nicola Sturgeon has certainly changed
PRINTING COMMENT RESULTS FOR THE TOPIC: 

The planet is in serious trouble,’ says Scotland’s leader, Nicola Sturgeon. [NY Times]
PRINTING COMMENT RESULTS FOR THE TOPIC: 

How do Scottish people view FMS Nicola Sturgeon and the English people &amp; why
PRINTING COMMENT RESULTS FOR THE TOPIC: 

Scotland's Nicola Sturgeon: 'We're Running out of Time." #Nicola #running #Scotlands #Sturgeon #time
PRINTING COMMENT RESULTS FOR THE TOPIC: 

‘The planet is in serious trouble,’ says Scotland’s lea

### Dictionary results

In [24]:
print(reddit_posts)

['COP26: Greta Thunberg tells protest that COP26 has been a failure', 'First Minister Nicola sturgeon is delighted to have received, on behalf of the people of Scotland, a gift from President Biden. She was so pleased to welcome President Biden to Glasgow, and looks forward to him returning to Scotland in the future. COP 26', 'Scottish First Minister Nicola Sturgeon has certainly changed', "**OP sent the following text as an explanation on why this is unexpected:**\n\n&gt;!It's the Krankies!&lt;\n\n*****\n**Is this an unexpected post with a fitting description?**\n**Then upvote this comment, otherwise downvote it.**\n\n*****\n[*Look at my source code on Github*](https://github.com/Artraxon/unexBot) [*What is this for?*](https://www.reddit.com/r/Unexpected/comments/dnuaju/introducing_unexbot_a_new_bot_to_improve_the/)", "It's okay, this was in Scottland. Anyone who might be watching doesn't give a fuck.", 'She might as well be one', 'Was this real?\nEasy enough mistake to make', 'The pl

## VADER Analysis test

In [ ]:
!pip install vaderSentiment

In [8]:
reddit_results = []
for post in reddit_posts.values():
   # print(post)
    reddit_results.extend(post)

print(reddit_results)

["Can't fault these answers. She makes most other politicians look like children.", "I usually can't listen to politicians because they're too embarrassing but she's good.", 'Like her or not and agree with independence or not, surely everyone can feel proud to have someone like this representing the UK right?\n\nWe are a laughing stock of the world with guys like Boris and Cameron, we need people that can actually give us credibility worldwide.\n\nThe sad thing is though people are so polarised they will still criticise her and praise Boris as some of the comments on this sub proves. Whatever happened to people being opened minded and able to give credit when someone is good and be against someone when its bad? Why do these Boris fanboys have to support him not matter what and be against Sturgeon no matter what, even when it makes them look like idiots. \n\nSad really.', "The full interview is 10 minutes long and the first half is about Sturgeon's hopes and concerns for COP26 , what de

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
# Example code:

f = open("reddit_scores.txt", "x", encoding="utf-8")

for reddit_post in reddit_results:
  vs = analyzer.polarity_scores(reddit_post)
  compound = vs['compound']
  sentiment_score = compound * 100
  print("The post '" + reddit_post + "' has a sentiment score of: " + (str(sentiment_score)) + "%")
  
  if (compound >= 0.05):
    print("The post was POSITIVE")
    f.write("\n" + reddit_post + ": RESULT = POSITIVE")
  elif (compound <= -0.05):
    print("The post was NEGATIVE")
    f.write("\n" + reddit_post + ": RESULT = NEGATIVE")
  else:
    print("The post was NEUTRAL")
    f.write("\n" + reddit_post + ": RESULT = NEUTRAL")
  print("{:-<65} {}".format(reddit_post, str(vs)))

f.close()